In [1]:
import os
import pandas as pd
import seaborn as sns
import scipy
import numpy as np
import random
from matplotlib import pyplot as plt
from datetime import datetime
from tqdm import tqdm
%matplotlib inline

from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore
from surprise.model_selection.validation import cross_validate
from sklearn.metrics import mean_squared_error, mean_absolute_error

import warnings
warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [3]:
user2idx = {user:idx for idx, user in enumerate(train['user_code'].unique())}
train['user_idx'] = train['user_code'].map(user2idx)
idx2user = {idx:user for idx, user in enumerate(train['user_code'].unique())}

In [4]:
item2idx = {item:idx for idx, item in enumerate(train['rest_code'].unique())}
train['rest_idx'] = train['rest_code'].map(item2idx)
idx2item = {idx:item for idx, item in enumerate(train['rest_code'].unique())}

In [5]:
test_user2idx = {user:idx for idx, user in enumerate(test['user_code'].unique())}
test['user_idx'] = test['user_code'].map(user2idx)
test_idx2user = {idx:user for idx, user in enumerate(test['user_code'].unique())}

In [6]:
test_item2idx = {item:idx for idx, item in enumerate(test['rest_code'].unique())}
test['rest_idx'] = test['rest_code'].map(item2idx)
test_idx2item = {idx:item for idx, item in enumerate(test['rest_code'].unique())}

In [7]:
train['rating'] = 1
test['rating'] = 1

In [8]:
train

,userid,rest,user_code,rest_code,user_idx,rest_idx,rating
0,5b61c7658f8242cb2a1b1028,1.100141e+09,0,14,0,0,1
1,5b61c7658f8242cb2a1b1028,3.049867e+08,0,6,0,1,1
2,5b61c7658f8242cb2a1b1028,1.757111e+09,0,17,0,2,1
3,5b61c7658f8242cb2a1b1028,1.372911e+09,0,8,0,3,1
4,5b61c7658f8242cb2a1b1028,1.347553e+09,0,12,0,4,1
...,...,...,...,...,...,...,...
6735896,63c3c2141438fc3a9c0ab5ad,3.139818e+07,382939,37021,382939,35796,1
6735897,63c3c2141438fc3a9c0ab5ad,3.348144e+07,382939,30508,382939,28309,1
6735898,63c3c2141438fc3a9c0ab5ad,1.010941e+09,382939,6918,382939,8192,1
6735899,63c3c2141438fc3a9c0ab5ad,1.015348e+09,382939,9479,382939,12639,1


In [9]:
from surprise import Reader, Dataset

reader = Reader(rating_scale=(0.5, 5.0))
train_data = Dataset.load_from_df(train[['user_idx', 'rest_idx', 'rating']], reader)

In [10]:
from surprise import Reader, Dataset

reader = Reader(rating_scale=(0.5, 5.0))
test_data = Dataset.load_from_df(test[['user_idx', 'rest_idx', 'rating']], reader)

In [11]:
sim_options = {
    'name': 'cosine',
    'user_based': True
}

knn_basic = KNNBasic(k=40, min_k = 10, sim_options=sim_options)

In [12]:
train_data = train_data.build_full_trainset()

In [13]:
type(train_data)

surprise.trainset.Trainset

In [14]:
knn_basic.fit(train_data)
predictions = knn_basic.test(test_data)

Computing the cosine similarity matrix...


MemoryError: Unable to allocate 1.07 TiB for an array with shape (382940, 382940) and data type float64